# Computação Gráfica e Realidade Aumentada 
## 14/11/2023
<author>Author: Manuel Santos</author>
<h1>Illumination Example</h1>

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

### Vertex Shader (Original)

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

### Fragment Shader (Original)

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void) {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

### Cube definition

In [ ]:
%%html

<script id="Cube1">
class cube extends CGRAobject
{
    // ===== Constructor
    constructor(glcontext, col1 = [1.0, 0.0, 0.0], col2 = col1)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Define COnstants
        this.numvertices = 36;
        this.size = 1.0;
        let dim = this.size/2.0;
        
        // ===== List vertices
        var verticesCube = [[ -dim, -dim,  dim],
                             [-dim,  dim,  dim],
                             [ dim,  dim,  dim],
                             [ dim, -dim,  dim],
                             [-dim, -dim, -dim],
                             [-dim,  dim, -dim],
                             [ dim,  dim, -dim],
                             [ dim, -dim, -dim]]; 

        // ===== Lookup table
        var vertices = [];
        var indexes = [
            0, 1, 2,  2, 3, 0,  // Front face
            6, 7, 3,  2, 3, 6,  // Right face
            6, 7, 4,  4, 5, 6,  // Back  face
            1, 0, 4,  4, 5, 1,  // Left  face
            1, 6, 2,  3, 4, 0,  // Up    face
            1, 6, 5,  3, 4, 7]; // Down  face
        
        // ===== Create structure to populate buffer
        for (var i = 0; i < indexes.length; i++)
        {
            vertices = vertices.concat(verticesCube[indexes[i]]);
        } 

        // ===== Assigning colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        
        // ===== Creating and configuring buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
}
</script>

### Sphere Definition

In [ ]:
%%html

<canvas id="SphereCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="SphereApp">

class sphere extends CGRAobject
{
    center = glm.vec4(0, 0, 0, 1); 
    north_pole = glm.vec4(0, 0.5, 0, 1); 
    
    // ===== Constructor
    constructor(glcontext, col = [1.0, 1.0, 1.0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        // ===== Number of vertices
        this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
        this.length = 1; this.radius = 0.5;
        
        // ===== Create circunferences (360/N sides)
        let slices = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides, aux = [];
            for (let j = 0; j < this.sides-1; j++)
            {     
                aux.push(
                    [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                     Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                    (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
            }   
            slices.push(aux);
        }
        slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
        
        
        var vertices = [];
        var colors = [];
        
        // ===== Create vertex and color data to populate buffers
        for (let i = 0; i < this.sides; i++)
        {            
            // Top side
            vertices.push(slices[this.sides][0]);
            vertices.push(slices[i][0]);
            vertices.push(slices[(i+1)%this.sides][0]);
            colors.push([col,col,col].flat());
            
            // Middle sections
            for (let j = 0; j < this.sides - 2; j++)
            {
                vertices.push(slices[i][j]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[i][j+1]);
                colors.push([col,col,col].flat());

                vertices.push(slices[i][j+1]);
                vertices.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[(i+1)%this.sides][j+1]);
                colors.push([col,col,col].flat());
            }
            
            // Down side
            vertices.push(slices[i][this.sides - 2]);
            vertices.push(slices[(i+1)%this.sides][this.sides - 2]);
            vertices.push(slices[this.sides][1]);
            colors.push([col,col,col].flat());
        }
        
        // ===== Populate Buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
    }
    
    // ===== Funcion to register center and north of sphere (WIP)
    update_pos(parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        //console.log(globalMat);
        
        this.center = globalMat['*'](this.center);
        //console.log("Center: " + this.center);
        this.north_pole = globalMat['*'](this.north_pole);
        //console.log("North: " + this.center);
    }
}


class SphereApp extends DEECapp
{
    counter=0;
    initialize()
    {
        // ===== Perform initializations
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9,0.9,0.9,1.0);
        
        // ===== Create sphere
        this.my_Sphere = new sphere(this.gl,[0.95, 0.95, 0]);
        this.my_Sphere.setShader(this.shaderprog);

        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,10);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(2, 1, 0): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(2,2,2),glm.vec3(0,0,0),glm.vec3(0,1,0));           
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        this.shaderprog.startUsing();
        
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.my_Sphere.setModelTransformation(model);
        
        // ===== Draw sphere
        this.my_Sphere.drawit(this.viewM,this.projectionM);
    }
}

var app = new SphereApp('SphereCanvas');

app.run();
</script>

## Exercises 7.1, 7.2 and 7.3

### Shaders for ambient lighting

In [ ]:
%%html
<script id="my-vertex-shader-ambient" type="x-shader/x-vertex">
precision mediump float;

// === Inputs
attribute  vec3 in_Position;
attribute  vec3 in_Color;
// === Outputs
varying  vec3 ex_Color;
// === Uniforms
uniform mat4 MVP;


void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-ambient" type="x-shader/x-fragment">
precision mediump float;

// === Inputs
//attribute vec3 in_Color;
// === Outputs
varying  vec3 ex_Color;
// === Uniforms
uniform vec4 Ambient; //sets lighting level, same across many vertices

void main(void)
{
    vec4 scatteredLight = Ambient; //this is the only light

    // modulate surface color with light, but saturate at white
    gl_FragColor = min(vec4(ex_Color, 1.0) * scatteredLight, vec4(1.0));
}
</script>

In [ ]:
%%html

<canvas id="AmbientCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="AmbientApp">
class AmbientApp extends DEECapp
{
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Initialize shaders
        // - Default Shader
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // - Ambient Shader
        var fragsrc_amb = document.getElementById("my-fragment-shader-ambient").text;
        var vertsrc_amb = document.getElementById("my-vertex-shader-ambient").text;
        this.shaderprog_amb = new DEECshader(this.gl);
        this.shaderprog_amb.srcShaders(vertsrc_amb,fragsrc_amb);
        
        this.light_Loc = this.gl.getUniformLocation(this.shaderprog_amb.shaderProgram, "Ambient");
        
        // ===== Instantiate cubes
        // - Default Shader
        this.myCube = new cube(this.gl, [1, 0, 0]);
        this.myCube.setShader(this.shaderprog);
        
        // - Ambient Shader
        this.myCube_amb = new cube(this.gl, [1, 0, 0]);
        this.myCube_amb.setShader(this.shaderprog_amb);


        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(glm.vec3(0,3,6),glm.vec3(0,0,0),glm.vec3(0,1,0));        
        
        
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
       
        // ============= No Light
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translations
        var translation = glm.vec3(1.0, 0.0, 0.0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCube.setModelTransformation(model); 
        
        this.shaderprog.startUsing();
        // ===== Draw Cube Amb
        this.myCube.drawit(this.viewM,this.projectionM);
        
        
        // ============= Ambient Light
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translations
        var translation = glm.vec3(-1.0, 0.0, 0.0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter*3);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCube_amb.setModelTransformation(model); 
        
        this.shaderprog_amb.startUsing();
        this.gl.uniform4f(this.light_Loc,0.5, 0.5, 0.5, 1.0);
        // ===== Draw Cube Def
        this.myCube_amb.drawit(this.viewM,this.projectionM);       
    }
}

var app = new AmbientApp('AmbientCanvas');

app.run();
</script>

## Exercises 7.4 and 7.5

### Shaders for directional lighting

In [ ]:
%%html
<script id="my-vertex-shader-direct" type="x-shader/x-vertex">
precision mediump float;

// === Inputs
attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 in_Normals;
// === Outputs
varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec4 Position;
// === Uniforms
uniform mat4 MVP;
uniform mat3 NormalMatrix;
uniform mat4 M;


void main(void)
{
    Normal = normalize(NormalMatrix * in_Normals);
    Position = M * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);  
    
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-direct" type="x-shader/x-fragment">
precision mediump float;

// === Outputs
varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec4 Position;
// === Uniforms
uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightDirection;
uniform vec3 ViewPos;
uniform float Shininess;
uniform float Strength;

void main(void)
{
    vec3 EyeDir = normalize(ViewPos - vec3(Position));
    
    vec3 HalfVector = normalize(LightDirection+EyeDir);
    
    float diffuse = max(0.0, dot(Normal,LightDirection));
    float specular = max(0.0, dot(Normal,HalfVector));
    
    if (diffuse <= 0.0)
        specular = 0.0;
    else
        specular = pow(specular, Shininess);
    
    vec3 scatteredLight = Ambient + LightColor * diffuse;
    vec3 reflectedLight = LightColor * specular * Strength;
    vec3 rgb = min(ex_Color * scatteredLight + reflectedLight, vec3(1.0));
    
    gl_FragColor = vec4(rgb,1.0);
}
</script>

In [ ]:
%%html

<script id="Cube2">
class cube_lighted extends CGRAobject
{
    constructor(glcontext, col1 = [1.0, 0.0, 0.0], col2 = col1)
    {
        // ===== Initialize the parent class
        super(glcontext); 
        
        // ===== Define Constants
        this.numvertices = 36;
        this.size = 1.0;
        let dim = this.size/2.0;
        
        // ===== List vertices
        var verticesCube = [[ -dim, -dim,  dim],
                             [-dim,  dim,  dim],
                             [ dim,  dim,  dim],
                             [ dim, -dim,  dim],
                             [-dim, -dim, -dim],
                             [-dim,  dim, -dim],
                             [ dim,  dim, -dim],
                             [ dim, -dim, -dim]]; 
        
        var normalsCube = [[ 0,  0,  1],  // Front = +Z
                           [ 1,  0,  0],  // Right = +X
                           [ 0,  0, -1],  // Back  = -Z
                           [-1,  0,  0],  // Left  = -X
                           [ 0,  1,  0],  // Up    = +Y
                           [ 0, -1,  0]]; // Down  = -Y

        // ===== Lookup table
        var vertices = [], normals = [];
        var indexes_vertices = [
            0, 1, 2,  2, 3, 0,  // Front face
            6, 7, 3,  2, 3, 6,  // Right face
            6, 7, 4,  4, 5, 6,  // Back  face
            1, 0, 4,  4, 5, 1,  // Left  face
            1, 5, 6,  6, 2, 1,  // Up    face
            0, 3, 7,  0, 4, 7]; // Down  face
        
        var indexes_normals = [
            0, 0, 0,  0, 0, 0,  // Front face
            1, 1, 1,  1, 1, 1,  // Right face
            2, 2, 2,  2, 2, 2,  // Back  face
            3, 3, 3,  3, 3, 3,  // Left  face
            4, 4, 4,  4, 4, 4,  // Up    face
            5, 5, 5,  5, 5, 5]; // Down  face
        
        
        // ===== Create structure to populate buffer
        for (var i = 0; i < indexes_vertices.length; i++)
        {
            vertices = vertices.concat(verticesCube[indexes_vertices[i]]);
            normals = normals.concat(normalsCube[indexes_normals[i]]);
        } 
        
        //console.log(vertices);
        //console.log(normals);
        
        // ===== Assigning colors
        var colors = [];
        this.color1 = [col1, col1, col2].flat();
        this.color2 = [col2, col2, col1].flat();
        for (var i=0; i < (this.numvertices/3)/2; i++)
        {
            colors.push(this.color1);
            colors.push(this.color2);
        }
        
        // ===== Creating and configuring buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.normalsbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(normals), this.gl.STATIC_DRAW);
    } 
    
    setparameters(Ambient, colorLight, positionLight, viewposition, shiny = 0.1, strength = 0.1)
    {    
        this.ambient = Ambient;
        this.colorLight = colorLight;
        this.positionLight = positionLight;
        this.viewposition = viewposition;
        this.Shininess = shiny;
        this.Strength = strength;
    }
    
    drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0))
    {    
        this.shaderprog.startUsing();
        
        this.normloc = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normloc, // Attribute location
                       3, // number of elements per attribute
                       this.gl.FLOAT,  // Type of elements
                       this.gl.FALSE,  // 
                       3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                        0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normloc);
        
        var NormMatloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        var localT = parentMat4['*'](this.modelMat); 
        var NormMat = glm.mat3(glm.transpose(glm.inverse(localT)));
        this.gl.uniformMatrix3fv(NormMatloc,false, NormMat.array);
        //console.log(NormMat.array)
        
        var Matloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"M");
        var localT = parentMat4['*'](this.modelMat); 
        this.gl.uniformMatrix4fv(NormMatloc,false, localT.array);
        
        var Ambientloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Ambient");
        this.gl.uniform3fv(Ambientloc, this.ambient.array);
        
        var LightColorloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(LightColorloc, this.colorLight.array);
        
        var LightDirloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDirection");
        this.gl.uniform3fv(LightDirloc, this.positionLight.array);
        
        var ViewPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPos");
        this.gl.uniform3fv(ViewPosloc, this.viewposition.array);
        
        var Shininessloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(Shininessloc, this.Shininess);
        
        var Strengthloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(Strengthloc, this.Strength);
        
        this.shaderprog.stopUsing();
        
        super.drawit(viewMat,projectionMat,parentMat4);
    }
}

</script>

In [ ]:
%%html

<canvas id="DirectCanvas_Cubes" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="DirectApp_Cubes">
class DirectApp_Cubes extends DEECapp
{
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Initialize shaders
        // - Default Shader
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // - Directional Shader
        var fragsrc_dir = document.getElementById("my-fragment-shader-direct").text;
        var vertsrc_dir = document.getElementById("my-vertex-shader-direct").text;
        this.shaderprog_dir = new DEECshader(this.gl);
        this.shaderprog_dir.srcShaders(vertsrc_dir,fragsrc_dir);
                
        var ambient_light = glm.vec3(0.5,0.5,0.7);
        var color_light = glm.vec3(1.0,1.0,1.0);
        var light_dir = glm.vec3(0.0, 0.0, 1.0);
        var viewpos = glm.vec3(0,3,6);
        
        // ===== Instantiate cubes
        // - Default Shader
        this.myCube = new cube(this.gl, [1, 0, 0]);
        this.myCube.setShader(this.shaderprog);
        
        // - Ambient Shader
        this.myCube_dir = new cube_lighted(this.gl);
        this.myCube_dir.setShader(this.shaderprog_dir);

        this.myCube_dir.setparameters(ambient_light, color_light, light_dir, viewpos, 0.8, 1);
        
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(viewpos,glm.vec3(0,0,0),glm.vec3(0,1,0));        
        
        
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
       
        // ============= No Light
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translations
        var translation = glm.vec3(1.0, 0.0, 0.0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCube.setModelTransformation(model); 
        
        // ===== Draw Cube 
        this.myCube.drawit(this.viewM,this.projectionM);
        
        
        // ============= Directional Light
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translations
        var translation = glm.vec3(-1.0, 0.0, 0.0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(0.0, 1.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.myCube_dir.setModelTransformation(model); 
        
        // ===== Draw Cube Dir
        this.myCube_dir.drawit(this.viewM,this.projectionM);       
    }
}

var app = new DirectApp_Cubes('DirectCanvas_Cubes');

app.run();
</script>

In [ ]:
%%html

<script id="Sphere2">

class sphere_lighted extends CGRAobject
{
    center = glm.vec4(0, 0, 0, 1); 
    north_pole = glm.vec4(0, 0.5, 0, 1); 
    
    // ===== Constructor
    constructor(glcontext, col = [0.95, 0.95, 0], faces = 10)
    {
        // Initialize the parent class
        super(glcontext); 
        
        // ===== Constants
        // Number of faces (3 to inf)
        this.sides = faces;
        // ===== Number of vertices
        this.numvertices = (6*this.sides + (6*this.sides)*(this.sides-2));
        this.length = 1; this.radius = 0.5;
        
        // ===== Create circunferences (360/N sides)
        let slices = [];   
        for (let i = 0; i < this.sides; i++)
        { 
            let theta = i*2*Math.PI/this.sides, aux = [];
            for (let j = 0; j < this.sides-1; j++)
            {     
                aux.push(
                    [(Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.cos(theta),
                     Math.cos((j+1)*Math.PI/this.sides)*this.radius,
                    (Math.sin((j+1)*Math.PI/this.sides)*this.radius)*Math.sin(theta)]);
            }   
            slices.push(aux);
        }
        slices.push([[0,this.radius,0],[0,-1*this.radius,0]]);
        
        
        var vertices = [];
        var colors = [];
        var normals = [];
        
        // ===== Create vertex and color data to populate buffers
        for (let i = 0; i < this.sides; i++)
        {            
            // Top side
            vertices.push(slices[this.sides][0]);
            normals.push(slices[this.sides][0]);
            vertices.push(slices[i][0]);
            normals.push(slices[i][0]);
            vertices.push(slices[(i+1)%this.sides][0]);
            normals.push(slices[(i+1)%this.sides][0]);
            colors.push([col,col,col].flat());
            
            // Middle sections
            for (let j = 0; j < this.sides - 2; j++)
            {
                vertices.push(slices[i][j]);
                normals.push(slices[i][j]);
                vertices.push(slices[(i+1)%this.sides][j]);
                normals.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[i][j+1]);
                normals.push(slices[i][j+1]);
                colors.push([col,col,col].flat());

                vertices.push(slices[i][j+1]);
                normals.push(slices[i][j+1]);
                vertices.push(slices[(i+1)%this.sides][j]);
                normals.push(slices[(i+1)%this.sides][j]);
                vertices.push(slices[(i+1)%this.sides][j+1]);
                normals.push(slices[(i+1)%this.sides][j+1]);
                colors.push([col,col,col].flat());
            }
            
            // Down side
            vertices.push(slices[i][this.sides - 2]);
            normals.push(slices[i][this.sides - 2]);
            vertices.push(slices[(i+1)%this.sides][this.sides - 2]);
            normals.push(slices[(i+1)%this.sides][this.sides - 2]);
            vertices.push(slices[this.sides][1]);
            normals.push(slices[this.sides][1]);
            colors.push([col,col,col].flat());
        }
        
        
        
        // ===== Populate Buffers
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices.flat()), this.gl.STATIC_DRAW);
        
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors.flat()), this.gl.STATIC_DRAW);
        
        this.normalsbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);    
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(normals.flat()), this.gl.STATIC_DRAW);
    }
    
    setparameters(Ambient, colorLight, positionLight, viewposition, shiny = 0.1, strength = 0.1)
    {    
        this.ambient = Ambient;
        this.colorLight = colorLight;
        this.positionLight = positionLight;
        this.viewposition = viewposition;
        this.Shininess = shiny;
        this.Strength = strength;
    }
    
    drawit(viewMat, projectionMat, parentMat4 = glm.mat4(1.0))
    {    
        this.shaderprog.startUsing();
        
        this.normloc = this.gl.getAttribLocation(this.shaderprog.shaderProgram,"in_Normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normloc, // Attribute location
                       3, // number of elements per attribute
                       this.gl.FLOAT,  // Type of elements
                       this.gl.FALSE,  // 
                       3*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                        0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normloc);
        
        var NormMatloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        var localT = parentMat4['*'](this.modelMat); 
        var NormMat = glm.mat3(glm.transpose(glm.inverse(localT)));
        this.gl.uniformMatrix3fv(NormMatloc,false, NormMat.array);
        //console.log(NormMat.array)
        
        var Matloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"M");
        var localT = parentMat4['*'](this.modelMat); 
        this.gl.uniformMatrix4fv(NormMatloc,false, localT.array);
        
        var Ambientloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Ambient");
        this.gl.uniform3fv(Ambientloc, this.ambient.array);
        
        var LightColorloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(LightColorloc, this.colorLight.array);
        
        var LightDirloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDirection");
        this.gl.uniform3fv(LightDirloc, this.positionLight.array);
        
        var ViewPosloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPos");
        this.gl.uniform3fv(ViewPosloc, this.viewposition.array);
        
        var Shininessloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(Shininessloc, this.Shininess);
        
        var Strengthloc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(Strengthloc, this.Strength);
        
        this.shaderprog.stopUsing();
        
        super.drawit(viewMat,projectionMat,parentMat4);
    }
    
    // ===== Funcion to register center and north of sphere (WIP)
    update_pos(parentMat4 = glm.mat4(1.0))
    {
        var globalMat = parentMat4['*'](this.modelMat);
        
        //console.log(globalMat);
        
        this.center = globalMat['*'](this.center);
        //console.log("Center: " + this.center);
        this.north_pole = globalMat['*'](this.north_pole);
        //console.log("North: " + this.center);
    }
}
</script>

In [ ]:
%%html

<canvas id="DirectCanvas_Spheres" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="DirectApp_Spheres">
class DirectApp_Spheres extends DEECapp
{
    counter = 0;
    initialize()
    {
        // ===== Perform initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.9, 0.9, 0.9, 1.0);
        
        // ===== Initialize shaders
        // - Default Shader
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // - Direct Shader
        var fragsrc_dir = document.getElementById("my-fragment-shader-direct").text;
        var vertsrc_dir = document.getElementById("my-vertex-shader-direct").text;
        this.shaderprog_dir = new DEECshader(this.gl);
        this.shaderprog_dir.srcShaders(vertsrc_dir,fragsrc_dir);
                
        var ambient_light = glm.vec3(0.5,0.5,0.7);
        var color_light = glm.vec3(0.3,0.3,0.3);
        var light_dir = glm.vec3(0.0, 2.0, 1.0);
        var viewpos = glm.vec3(0,2,4);
        
        // ===== Instantiate cubes
        // - Default Shader
        this.mySphere = new sphere(this.gl, [0.0, 0.15, 0.85], 30);
        this.mySphere.setShader(this.shaderprog);
        
        // - Ambient Shader
        this.mySphere_dir = new sphere_lighted(this.gl, [0.0, 0.15, 0.85], 30);
        this.mySphere_dir.setShader(this.shaderprog_dir);

        this.mySphere_dir.setparameters(ambient_light, color_light, light_dir, viewpos, 5, 2);
        
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        //glm.radians(45): Esse argumento especifica o campo de visão em graus. Aqui, você está usando um campo de visão de 45 graus.             //       
        //1: A razão de aspecto (aspect ratio) da janela ou tela. No seu caso, a razão de aspecto é 1, o que significa que a janela é um quadrado.//
        //0.1: O valor mais próximo em relação à câmera em coordenadas de profundidade.                                                           //
        //1000: O valor mais distante em relação à câmera em coordenadas de profundidade.                                                         // 
        ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        this.projectionM= glm.perspective(glm.radians(45),1.0,0.1,100);      
        
        //////////////////////////////////////////////////////////////////////////
        //glm.vec3(3, 3, 5): A posição da câmera no espaço 3D.                  //
        //glm.vec3(0, 0, 0): O ponto para o qual a câmera está direcionada.    //
        //glm.vec3(0, 1, 0): A orientação da "parte de cima" da câmera.         //
        //////////////////////////////////////////////////////////////////////////
        this.viewM = glm.lookAt(viewpos,glm.vec3(0,0,0),glm.vec3(0,1,0));         
    }
    
    render()
    {
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
       
        // ============= No Light
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translations
        var translation = glm.vec3(0.75, 0.0, 0.0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(1.0, 0.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.mySphere.setModelTransformation(model); 
        
        this.shaderprog.startUsing();
        // ===== Draw Cube 
        this.mySphere.drawit(this.viewM,this.projectionM);
        
        
        // ============= Directional Light
        // ===== Model
        var model = glm.mat4(glm.mat3(1.0));
        // ===== Translations
        var translation = glm.vec3(-0.75, 0.0, 0.0);
        model = glm.translate(model, translation);
        // ===== Rotation
        var rotation = glm.radians(this.counter);
        var axis = glm.vec3(1.0, 0.0, 0.0);
        model = glm.rotate(model, rotation, axis);
        // ===== Final Transformation
        this.mySphere_dir.setModelTransformation(model); 
        
        this.shaderprog_dir.startUsing();
        // ===== Draw Cube Dir
        this.mySphere_dir.drawit(this.viewM,this.projectionM);   
        
        
        //this.viewM = glm.lookAt(glm.vec3(2*Math.sin(this.counter/20),2,4),glm.vec3(0,0,0),glm.vec3(0,1,0));
    }
}

var app = new DirectApp_Spheres('DirectCanvas_Spheres');

app.run();
</script>